## DarkSky API pull for fire post clustered data
### Main columns needed are latitude, longitude, datetime

In [50]:
import pandas as pd
import datetime
import numpy as np

import config

from darksky import forecast
from datetime import datetime as dt
from datetime import timedelta
from datetime import date
from sqlalchemy import create_engine

In [51]:
# create sqlalchemy engine
engine = create_engine("mysql+mysqlconnector://{user}:{password}@{host}/{dbname}"
                       .format(user=config.db_user,
                               password=config.db_pass,
                               dbname=config.db_name,
                               host=config.db_host))

In [52]:
RAPIDAPI_KEY  = config.darksky_api1
RAPIDAPI_KEY2  = config.darksky_api2

In [53]:
def weather_lookup(df, key=RAPIDAPI_KEY, days_before=0, days_after=0):
    data = []
    for index, row in df.iterrows():
        ts = row['datetime'].isoformat() 
        lat = row['latitude']
        lon = row['longitude']
        weather = forecast(key, lat, lon, time=ts)
        w_dict = weather['currently']
        w_dict['timestamp'] = ts
        w_dict['latitude'] = lat
        w_dict['longitude'] = lon
        data.append(w_dict)
    return data


In [97]:
filename = "fire_lat_lon_data/Emissions2015_DBScan_Clusters.csv"

In [98]:
firedata = pd.read_csv(filename)

In [99]:
firedata['datetime'] = (pd.to_datetime(firedata.year, format='%Y') + 
                       firedata['doy'].apply(np.ceil).apply(
                           lambda x: pd.Timedelta(x, unit='D')) + timedelta(hours=12))

In [100]:
firedata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8588 entries, 0 to 8587
Data columns (total 24 columns):
Unnamed: 0             8588 non-null int64
id                     8588 non-null float64
year                   8588 non-null float64
doy                    8588 non-null float64
longitude              8588 non-null float64
latitude               8588 non-null float64
grid10k                8588 non-null float64
covertype              8588 non-null float64
fuelcode               8588 non-null float64
area_burned            8588 non-null float64
prefire_fuel           8588 non-null float64
consumed_fuel          8588 non-null float64
ECO2                   8588 non-null float64
ECO                    8588 non-null float64
ECH4                   8588 non-null float64
EPM2.5                 8588 non-null float64
cwd_frac               8588 non-null float64
duff_frac              8588 non-null float64
fuel_moisture_class    8588 non-null float64
burn_source            8588 non-null flo

In [101]:
firedata1000 = firedata[:]

In [102]:
firedata2000 = firedata[1000:2000]

In [103]:
firedata3000 = firedata[2000:]

In [104]:
weather_df1.head()

,apparentTemperature,cloudCover,dewPoint,humidity,icon,latitude,longitude,precipAccumulation,precipIntensity,precipProbability,...,pressure,summary,temperature,time,timestamp,uvIndex,visibility,windBearing,windGust,windSpeed
0,68.24,0.0,59.88,0.75,clear-day,48.7142,-121.9806,NaN,0.0000,0.0,...,1018.9,Clear,68.21,1405450800,2014-07-15T12:00:00,9.0,10.000,209.0,2.44,2.17
1,53.56,NaN,53.43,1.00,NaN,48.9251,-120.4474,NaN,0.0116,0.2,...,NaN,NaN,53.43,1407956400,2014-08-13T12:00:00,NaN,NaN,175.0,6.62,2.29
2,63.40,0.0,52.90,0.69,clear-day,48.1125,-124.6477,NaN,0.0000,0.0,...,1022.2,Clear,63.40,1407610800,2014-08-09T12:00:00,7.0,7.995,300.0,9.14,6.10
3,64.03,NaN,62.75,0.98,NaN,48.6051,-121.5700,NaN,0.0000,0.0,...,NaN,NaN,63.40,1407870000,2014-08-12T12:00:00,NaN,NaN,43.0,3.59,1.33
4,69.67,NaN,39.74,0.34,NaN,48.7550,-120.2527,NaN,0.0000,0.0,...,NaN,NaN,69.67,1407351600,2014-08-06T12:00:00,NaN,NaN,316.0,9.94,3.89


In [105]:
weather_df1 = pd.DataFrame(weather_lookup(firedata1000,RAPIDAPI_KEY))

In [106]:
#weather_df2 = pd.DataFrame(weather_lookup(firedata4000,RAPIDAPI_KEY2))

In [107]:
db_columns = ['apparentTemperature', 'cloudCover', 'dewPoint',
       'humidity', 'icon', 'latitude', 'longitude', 'precipAccumulation',
       'precipIntensity', 'precipProbability', 'precipType', 'pressure',
       'summary', 'temperature', 'time', 'timestamp', 'uvIndex', 'visibility',
       'windBearing', 'windGust', 'windSpeed']

In [108]:
weather_df1 = weather_df1[db_columns]

In [109]:
#weather_df2 = weather_df2[db_columns]

In [111]:
weather_df1.to_sql('weather_loading', con = engine, if_exists = 'append', chunksize = 100000)

In [112]:
#weather_df2.to_sql('weather_loading', con = engine, if_exists = 'append', chunksize = 100000)

In [113]:
query = """SELECT * FROM weather_loading;
"""

In [114]:
weather_all_sql = pd.read_sql(query, engine)

In [115]:
#weather_all_sql = weather_all_sql.reset_index()[db_columns]

In [116]:
#weather_all_sql.to_sql('weather_loading', con = engine, if_exists = 'replace', chunksize = 100000)

In [117]:
weather_all_sql.to_csv('weather_80000.csv')

In [118]:
weather_all_sql.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79283 entries, 0 to 79282
Data columns (total 22 columns):
index                  79283 non-null int64
apparentTemperature    78565 non-null float64
cloudCover             63486 non-null float64
dewPoint               78182 non-null float64
humidity               78167 non-null float64
icon                   63545 non-null object
latitude               79283 non-null float64
longitude              79283 non-null float64
precipAccumulation     409 non-null float64
precipIntensity        76491 non-null float64
precipProbability      76491 non-null float64
precipType             9021 non-null object
pressure               47953 non-null float64
summary                64018 non-null object
temperature            78565 non-null float64
time                   79283 non-null int64
timestamp              79283 non-null object
uvIndex                63486 non-null float64
visibility             65172 non-null float64
windBearing            77722

In [119]:
weather_df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8588 entries, 0 to 8587
Data columns (total 21 columns):
apparentTemperature    8561 non-null float64
cloudCover             7240 non-null float64
dewPoint               8532 non-null float64
humidity               8529 non-null float64
icon                   7367 non-null object
latitude               8588 non-null float64
longitude              8588 non-null float64
precipAccumulation     38 non-null float64
precipIntensity        8248 non-null float64
precipProbability      8248 non-null float64
precipType             878 non-null object
pressure               5390 non-null float64
summary                7425 non-null object
temperature            8561 non-null float64
time                   8588 non-null int64
timestamp              8588 non-null object
uvIndex                7240 non-null float64
visibility             7420 non-null float64
windBearing            8494 non-null float64
windGust               8287 non-null float64
wi